In [ ]:
import os
import numpy as np
from scipy.io import wavfile
from scipy.signal import lfilter

In [ ]:
# Function to apply a comb filter to audio data
def apply_comb_filter(data, fs, w0, gain):
    delay = int(fs / w0)  # Calculate the delay in samples
    feedback_gain = gain  # Feedback gain for the comb filter
    comb_filter = np.zeros(delay + 1)
    comb_filter[0] = 1

    # Apply the comb filter using a difference equation
    filtered_data = lfilter([1], comb_filter, data)
    filtered_data[delay:] -= feedback_gain * filtered_data[:-delay]

    return filtered_data

In [ ]:
# Function to process audio files and generate dataset
def create_dataset(audio_folder, target_folder, w_values=[100, 200, 300, 400, 500], g_values=[0.2, 0.4, 0.6, 0.8, 1.0]):
    # Create target folder if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # List all audio files in the input folder
    audio_files = os.listdir(audio_folder)

    for audio_file in audio_files:
        # Load the audio file
        audio_path = os.path.join(audio_folder, audio_file)
        fs, audio_data = wavfile.read(audio_path)

        for i, (w, gain) in enumerate(zip(w_values, g_values)):
            # Apply the comb filter with gain to the audio data
            filtered_data = apply_comb_filter(audio_data, fs, w, gain)

            # Save the filtered data to a new audio file
            target_path = os.path.join(target_folder, f'{audio_file}_{i}.wav')
            wavfile.write(target_path, fs, filtered_data)

            # Save the w0 and gain values as targets for the neural network
            target_values = np.array([w, gain])
            target_file = f'{audio_file}_{i}.npy'
            target_path = os.path.join(target_folder, target_file)
            np.save(target_path, target_values)

        print(f'Processed: {audio_file}')

In [ ]:
# Set the paths for the audio folder and target folder
audio_folder = 'path/to/audio/folder'
target_folder = 'path/to/target/folder'

# Create the dataset
create_dataset(audio_folder, target_folder)